In [2]:
import numpy as np
np.__version__

'2.3.2'

In [3]:
# 5x5 input image
image = np.array([
    [3, 0, 1, 2, 7],
    [1, 5, 8, 9, 3],
    [2, 7, 2, 5, 1],
    [0, 1, 3, 1, 7],
    [4, 2, 1, 6, 2]
])
#image = np.random.randint(10, size=(6,6))

# 3x3 kernel
kernel = np.array([
    [1, 0, 1],
    [0, 1, 0],
    [1, 0, -1]
])

image, kernel

(array([[3, 0, 1, 2, 7],
        [1, 5, 8, 9, 3],
        [2, 7, 2, 5, 1],
        [0, 1, 3, 1, 7],
        [4, 2, 1, 6, 2]]),
 array([[ 1,  0,  1],
        [ 0,  1,  0],
        [ 1,  0, -1]]))

In [4]:
image.shape, kernel.shape

((5, 5), (3, 3))

### Manual Convolution

In [5]:
def conv2d(img, kernel):
    H,W = img.shape
    k = kernel.shape[0]
    out = np.zeros((H-k+1, W-k+1))

    for i in range(H-k+1):
        for j in range(W-k+1):
            region = img[i:i+k, j:j+k]
            out[i, j] = np.sum(region * kernel)
    return out

conv_out = conv2d(image, kernel)
print('Conv Output:\n', conv_out)

Conv Output:
 [[ 9. 12. 18.]
 [13. 16. 12.]
 [ 8. 11.  3.]]


### ReLU layer

In [6]:
relu_out = np.maximum(0, conv_out)
print("\nReLU output:\n",relu_out)


ReLU output:
 [[ 9. 12. 18.]
 [13. 16. 12.]
 [ 8. 11.  3.]]


### Padding

In [7]:
def padding(img):
    H, W = img.shape
    if H % 2 != 0:
        img = np.vstack([img, np.zeros((1, W))])
    if W%2 != 0:
        img = np.hstack([img, np.zeros((img.shape[0], 1))])    
    return img

relu_out_padded = padding(relu_out)
relu_out_padded

array([[ 9., 12., 18.,  0.],
       [13., 16., 12.,  0.],
       [ 8., 11.,  3.,  0.],
       [ 0.,  0.,  0.,  0.]])

### Pooling layer

In [8]:
def max_pool(img):
    H, W = img.shape
    out = np.zeros((H//2, W//2))
    for i in range(0, H, 2):
        for j in range(0, W, 2):
            out[i//2, j//2] = np.max(img[i:i+2, j:j+2])
    return out

pool_out = max_pool(relu_out_padded)
print("\nPool output:\n", pool_out)


Pool output:
 [[16. 18.]
 [11.  3.]]


## Flatten

In [9]:
flat = pool_out.flatten()
print(flat)

[16. 18. 11.  3.]


### Fully Connected Layer

In [10]:
W = np.random.randn(3, len(flat))
b = np.random.randn(3)
print(W)
print(b)

[[-0.77086715  0.40549472  0.09104858 -0.46789995]
 [-0.24543572 -0.23878101  0.56562365 -0.0494905 ]
 [-1.37830891  1.36152909 -1.52459152  0.51720964]]
[-0.49620807 -0.31645532 -0.26923962]


In [11]:
W.shape

(3, 4)

In [12]:
fc_out = W @ flat + b
print("\nFC output:", fc_out)


FC output: [ -5.933343    -2.4680963  -13.03353639]


In [13]:
probs = np.exp(fc_out) / np.sum(np.exp(fc_out))
print("\nSoftmax probabilities:", probs)


Softmax probabilities: [3.03166505e-02 9.69658340e-01 2.50095805e-05]
